Copyright (c) 2024 Houssem Ben Braiek, Emilio Rivera-Landos, IVADO, SEMLA

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
#LOAD ENV VARIABLES
import os
from dotenv import load_dotenv
load_dotenv()
MLFLOW_URI = os.getenv("MLFLOW_URI")

In [2]:
%load_ext autoreload
%autoreload 2
# TODO: comment out if you want to check info logs
# import logging
# logging.basicConfig(level=logging.INFO)
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import sys,os
from lightgbm import LGBMRegressor

In [3]:
# Get the current directory path
current_directory = os.getcwd()

# Get the grandparent directory
grandparent_dir = os.path.abspath(os.path.join(current_directory, '..', 'src'))
sys.path.append(grandparent_dir)
# Add the parent directory to sys.path
# parent_dir = os.path.join(grandparent_dir, 'flight_delay')
# sys.path.append(parent_dir)

# sys.path.append(os.path.join(parent_dir, 'mlflow2'))



In [5]:
from flight_delay import pre_processing,train_utils,features

In [6]:
import flight_delay.mlflow.registry as registry
import flight_delay.mlflow.tracking as tracking

In [7]:
# # TODO(PARTICIPANT): Update IF NEEDED the data_dir
# dataset, dvc_info = load_dataset_from_dvc(data_dir="../data")

#load the data
data_path = '../data/flights_delay_dataset.csv'
df_flights = pre_processing.load_data(data_path)

In [8]:
df = df_flights[df_flights["covid_data"] == 0]

train_start = "2019-01-01"
train_end   = "2023-01-01"

test_start = "2023-01-01"
test_end   = "2023-09-01"

target_col = "ARR_DELAY"
split_col  = "FL_DATE"

X_train,y_train,X_test,y_test = train_utils.train_test_split_func(df,train_start,train_end,test_start,test_end,split_col,target_col)

In [9]:
numerical_cols = features.numerical_cols
categorical_cols = features.categorical_cols
time_cols = features.time_cols
passthrough_cols = features.passthrough_cols

features_list = numerical_cols + categorical_cols + time_cols + passthrough_cols

predicted_var = "predicted_arrival_delay"

#### Creating info dictionaries

Notice we add information about the DVC file:

In [10]:
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
mlflow.set_tracking_uri(MLFLOW_URI)

from flight_delay.mlflow.tracking import Experiment

## Iteration 1
In this initial iteration, we prepare the dataset for binary classification modeling and proceed to test our first baseline model, the Decision Tree, using our library.

In [11]:
experiment_name = 'lightGBM'
experiment = Experiment(MLFLOW_URI, experiment_name)

In [12]:
X_train,y_train,X_test,y_test = train_utils.train_test_split_func(df,train_start,train_end,test_start,test_end,split_col,target_col)

In [13]:
results = train_utils.train_model_with_tracking(experiment,LGBMRegressor(),LGBMRegressor.__name__, X_train, y_train,X_test,y_test,categorical_cols, numerical_cols, time_cols, passthrough_cols)

/home/ivamlops/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020908 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1919
[LightGBM] [Info] Number of data points in the train set: 1409857, number of used features: 774
[LightGBM] [Info] Start training from score 6.068654


/home/ivamlops/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/ivamlops/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/ivamlops/.local/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing 

🏃 View run run_LGBMRegressor_b8ba660a at: https://mlflow.p02-17.labs.ivamlops.ca/#/experiments/1/runs/4150aba82c414c01bf0fabb1b7cd86af
🧪 View experiment at: https://mlflow.p02-17.labs.ivamlops.ca/#/experiments/1


In [14]:
results

[{'regressor': 'LGBMRegressor',
  'MAE_train': 24.114304280561978,
  'MAE_test': 26.803231543466417}]

In [15]:
best_run = tracking.get_best_run(experiment)

## Final Step
It is important now to register our best advanced model into model registry. We will show how to transition to staging, as well as promoting to production using bank-marketing library, however, these steps should be done after careful validation.

In [16]:
model_name = "lightGBM"

In [17]:
model_version = registry.register_model_from_run(experiment.tracking_server_uri, best_run, model_name)

Successfully registered model 'lightGBM'.
2025/02/03 15:27:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lightGBM, version 1
Created version '1' of model 'lightGBM'.


In [18]:
registry.transition_model_to_staging(experiment.tracking_server_uri, model_name, model_version.version)

In [19]:
model_version = registry.promote_model_to_production(experiment.tracking_server_uri, model_name, f"{model_name}-production")

Successfully registered model 'lightGBM-production'.
Copied version '1' of model 'lightGBM' to version '1' of model 'lightGBM-production'.


In [20]:
registry.set_active_production_model(experiment.tracking_server_uri,  f"{model_name}-production", model_version.version)